In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import time
from collections import OrderedDict

from collections import namedtuple
from itertools import product

from torch.utils.tensorboard import SummaryWriter
from IPython.display import display , clear_output
import pandas as pd
import json
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
train_set  = torchvision.datasets.FashionMNIST(
root = './',
    train=True,
    download = True,
    transform  = transforms.Compose([
        transforms.ToTensor()
    ])
)

In [6]:
loader = DataLoader(train_set,batch_size=len(train_set),num_workers=1)
data = next(iter(loader))
mean , std  =data[0].mean() , data[0].std()

In [28]:
train_set_normal  = torchvision.datasets.FashionMNIST(
root = './',
    train=True,
    download = True,
    transform  = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean,std)
    ])
)

In [31]:
loader = DataLoader(train_set_normal,batch_size=1000,num_workers=1)
num_of_pixels = len(train_set)*28*28

total_sum = 0
for batch in loader:
    total_sum += batch[0].sum()
mean = total_sum/num_of_pixels

sum_of_squared_error = 0
for batch in loader:
    sum_of_squared_error += ((batch[0]-mean).pow(2)).sum()
std  = torch.sqrt(sum_of_squared_error/num_of_pixels)

mean,std

(tensor(1.5166e-06), tensor(1.0000))

In [32]:
print("Mean : " ,mean)
print("Standard deviation : " , std)

Mean :  tensor(1.5166e-06)
Standard deviation :  tensor(1.0000)


In [33]:
trainsets = {
    'not_normal':train_set,
    'normal': train_set_normal
}

In [34]:
class RunManager():
    def __init__(self):
        self.epoch_count = 0
        self.epoch_loss  = 0
        self.epoch_num_correct = None
        
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        self.network = None
        self.loader = None
        self.tb = None
    
    def begin_run(self,run,network,loader):
        self.run_start_time = time.time()
        self.run_params = run
        self.run_count += 1
        
        self.network = network
        self.loader  =loader
        self.tb  = SummaryWriter(comment=f'--{run}')
        
        images,labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)
        
        self.tb.add_image('images',grid)
        self.tb.add_graph(self.network,images.to(getattr(run,'device','cpu')))
        
    def end_run(self):
        self.tb.close()
        self.epoch_count = 0
        
    def begin_epoch(self):
        self.epoch_start_time  = time.time()
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
    
    def end_epoch(self):
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time
        
        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy  = self.epoch_num_correct / len(self.loader.dataset)
        
        self.tb.add_scalar('Loss',loss,self.epoch_count)
        self.tb.add_scalar('Accuracy',accuracy,self.epoch_count)
        
        for name , param in self.network.named_parameters():
            self.tb.add_histogram(name,param,self.epoch_count)
            self.tb.add_histogram(f'{name}.grad',param.grad,self.epoch_count)
            
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        
        for k,v in self.run_params._asdict().items(): results[k] = v
        self.run_data.append(results)
        df = pd.DataFrame.from_dict(self.run_data,orient="columns")
        clear_output(wait=True)
        display(df)
        
    def track_loss(self,loss):
        self.epoch_loss += loss.item() * self.loader.batch_size
    
    def track_num_correct(self,preds,labels):
        self.epoch_num_correct = self._get_num_correct(preds,labels)
    
    @torch.no_grad()
    def _get_num_correct(self,preds,labels):
        return preds.argmax(dim=1).eq(labels).sum().item()
    
    def save(self,fileName):
        pd.DataFrame.from_dict(self.run_data,orient='columns').to_csv(f'{fileName}.csv')
        with open(f'{fileName}.json','w+',encoding="utf-8") as f:
            json.dump(self.run_data,f,ensure_ascii=False,indent=4)
            
        

In [35]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4,out_features=120)
        self.fc2 = nn.Linear(in_features = 120 , out_features = 60 )
        self.out = nn.Linear(in_features=60 , out_features=10)
        
    def forward(self,t):
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t,kernel_size=2,stride=2)
        
        t = self.conv2(t)
        t  = F.relu(t)
        t = F.max_pool2d(t,kernel_size=2,stride=2)
        
        t =  t.reshape(-1,12*4*4)
        t = self.fc1(t)
        t = F.relu(t)
        
        t = self.fc2(t)
        t = F.relu(t)
        
        t = self.out(t)
        return t

In [36]:
class RunBuilder():
    @staticmethod
    def get_runs(params):
        Run = namedtuple('Run',params.keys())
        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))
        return runs

In [52]:
#With Run manager
params = OrderedDict(
    lr = [.01],
    batch_size = [1000],
    num_workers = [1],
    device = ['cuda'],
    trainset  = ['not_normal','normal']
    
)
m = RunManager()
for run in RunBuilder.get_runs(params):
    device = torch.device(run.device)
    network = Network().to(device)
    loader = DataLoader(trainsets[run.trainset],batch_size=run.batch_size,num_workers=run.num_workers)
    optimizer = optim.Adam(network.parameters(),lr=run.lr)
    m.begin_run(run,network,loader)
    for epoch in range(2):
        m.begin_epoch()
        for batch in loader:
            images = batch[0].to(device)
            labels  = batch[1].to(device)
            preds  = network(images)
            loss =  F.cross_entropy(preds,labels)
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss)
            m.track_num_correct(preds,labels)
        
        m.end_epoch()
    m.end_run()
m.save('results') 

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,num_workers,device,trainset
0,1,1,2.268222,0.001500,4.172163,5.233207,0.01,1000,1,cuda,not_normal
1,1,2,2.342318,0.001717,4.200280,9.483493,0.01,1000,1,cuda,not_normal
2,2,1,2.211853,0.001900,6.070232,7.199132,0.01,1000,1,cuda,normal
3,2,2,2.375262,0.001667,6.034155,13.289290,0.01,1000,1,cuda,normal


In [54]:
# x  = tuple([float(mean),])

In [39]:
data[0][0].shape

torch.Size([1, 28, 28])

In [45]:
time.time()

AttributeError: 'builtin_function_or_method' object has no attribute 'time'